# Pytorch图像分类模型转ONNX-ImageNet1000类

把Pytorch预训练ImageNet图像分类模型，导出为ONNX格式，用于后续在推理引擎上部署。

代码运行云GPU平台：公众号 人工智能小技巧 回复 gpu

同济子豪兄 2022-8-22 2023-4-28 2023-5-8

## 导入工具包

In [4]:
import torch
from torchvision import models
from torch import nn
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 载入ImageNet预训练PyTorch图像分类模型

In [5]:
class CustomMobileNetV3(nn.Module):
    def __init__(self, num_classes=5):
        super(CustomMobileNetV3, self).__init__()
        self.base_model = models.mobilenet_v3_small(pretrained=True)
        # 修改最后一层全连接层的输出节点数，以适应五分类任务
        in_features = self.base_model.classifier[-1].in_features
        self.base_model.classifier[-1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)
file_path = "0.8163_best_MobileNet_translearning.pth"
model_MobileNet =  CustomMobileNetV3()
model_MobileNet.load_state_dict(torch.load(file_path))
model_name_MobileNet = 'MobileNet_translearning'
model_MobileNet(torch.randn(16,3,256,256)).shape

model =  model_MobileNet
model = model.eval().to(device)

## 构造一个输入图像Tensor

In [6]:
x = torch.randn(1, 3, 256, 256).to(device)

## 输入Pytorch模型推理预测，获得1000个类别的预测结果

In [7]:
output = model(x)

In [8]:
output.shape

torch.Size([1, 5])

## Pytorch模型转ONNX格式

In [9]:
with torch.no_grad():
    torch.onnx.export(
        model,                       # 要转换的模型
        x,                           # 模型的任意一组输入
        f'{model_name_MobileNet}.onnx',    # 导出的 ONNX 文件名
        opset_version=11,            # ONNX 算子集版本
        input_names=['input'],       # 输入 Tensor 的名称（自己起名字）
        output_names=['output']      # 输出 Tensor 的名称（自己起名字）
    ) 

## 验证onnx模型导出成功

In [10]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load(f'{model_name_MobileNet}.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


## 以可读的形式打印计算图

In [11]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %base_model.features.1.block.1.fc1.weight[FLOAT, 8x16x1x1]
  %base_model.features.1.block.1.fc1.bias[FLOAT, 8]
  %base_model.features.1.block.1.fc2.weight[FLOAT, 16x8x1x1]
  %base_model.features.1.block.1.fc2.bias[FLOAT, 16]
  %base_model.features.4.block.2.fc1.weight[FLOAT, 24x96x1x1]
  %base_model.features.4.block.2.fc1.bias[FLOAT, 24]
  %base_model.features.4.block.2.fc2.weight[FLOAT, 96x24x1x1]
  %base_model.features.4.block.2.fc2.bias[FLOAT, 96]
  %base_model.features.5.block.2.fc1.weight[FLOAT, 64x240x1x1]
  %base_model.features.5.block.2.fc1.bias[FLOAT, 64]
  %base_model.features.5.block.2.fc2.weight[FLOAT, 240x64x1x1]
  %base_model.features.5.block.2.fc2.bias[FLOAT, 240]
  %base_model.features.6.block.2.fc1.weight[FLOAT, 64x240x1x1]
  %base_model.features.6.block.2.fc1.bias[FLOAT, 64]
  %base_model.features.6.block.2.fc2.weight[FLOAT, 240x64x1x1]
  %base_model.features.6.block.2.fc2.bias[FLOAT, 240]
  %bas

limit_output extension: Maximum message size of 10000 exceeded with 14908 characters

## 使用Netron可视化模型结构

Netron：https://netron.app

视频教程：https://www.bilibili.com/video/BV1TV4y1P7AP